<h1>EECS 549 Final Project</h1>

Data exploration & basic mainipulation

In [1]:
import os 
import pandas as pd
import json
import csv
from tqdm import tqdm
import numpy as np

In [2]:
np.random.seed(42)

In [3]:
posts = pd.read_csv("../data/ten-million-reddit-answers-posts.csv")

comments = pd.read_csv("../data/ten-million-reddit-answers-comments.csv")

<h2>Combine datasets</h2>

In [4]:
posts['created_date'] = pd.to_datetime(posts['created_utc'], unit = 's')

In [5]:
comments['post permalink'] = comments['permalink'].apply(lambda s: "/".join(s.split("/")[:-2]) + "/")

In [6]:
#prepare data
posts_subset = posts[['title', 'permalink', 'created_date']]
posts_subset = posts_subset.rename(columns = {'created_date': 'post_created'})
comments = comments.drop(columns = ['permalink'])

merge = posts_subset.merge(comments, how = 'left', left_on = 'permalink', right_on = 'post permalink')

In [7]:
merge.shape

(9999980, 13)

In [8]:
merge.groupby('title')['body'].nunique().agg(['mean', 'median', 'min', 'max'])

mean         15.300088
median        4.000000
min           0.000000
max       51673.000000
Name: body, dtype: float64

<h2>Summary statistics about post dates</h2>

In [9]:
posts['created_date'].agg(['min', 'max'])

min   2010-02-18 22:44:06
max   2020-11-30 23:59:39
Name: created_date, dtype: datetime64[ns]

<h2>Create new dataset made up of full posts</h2>

In [10]:
# new numeric index 
merge = merge.reset_index()
merge = merge.drop(columns = ["id"])
merge = merge.rename(columns = {'index': 'docid'})

In [11]:
merge = merge.sort_values(by = 'created_utc', ascending = True)

In [ ]:
merge.to_csv("../data/ask_reddit.csv", index = False)

In [ ]:
titles = list(merge['title'].astype(str).values)
comments = list(merge['body'].astype(str).values)
idx = list(merge['docid'].values)

In [ ]:
used_idx = np.random.choice(idx, size = 300000, replace = False)

In [ ]:
idx_to_post = {}

for i in tqdm(range(len(used_idx))):
    docid = idx[i]
    title = titles[i]
    comment = comments[i]
    if '[deleted]' in comment or '[removed]' in comment or '**PLEASE READ THIS MESSAGE IN ITS ENTIRETY BEFORE TAKING ACTION.*' in comment: 
        continue
    if id not in list(idx_to_post.values()):
        idx_to_post[docid] = title + " " + comment + " "
        continue
    curr = idx_to_post[docid]
    if len(curr) <= 30000:
        idx_to_post[docid] = idx_to_post[docid] + comment + " "

<h2>Write dataset to file</h2>

In [ ]:
with open("../data/ask_reddit_posts_v2.jsonl", "w") as f:
    for k, v in list(idx_to_post.items()):
        d = {'docid': k, 'text': v}
        print(json.dumps(d), file = f)